### Model API Workflow

1. CloudWatch Events로부터 호출 받을 때, 시간을 보내거나, image 내에서 현재 시간 얻어내거나. (난이도로 봤을 때 훨씬 쉬운건 아무것도 안보내고, image 내에서 진행하는 것.)

2. 시간을 20분 단위로 내림하기.

3. RDS로부터 내림한 시간을 기준으로 데이터 받음. (evs info & timestamp's sequence(length: n) & pymysql)

4. 받은 데이터 utils.py 돌리고 model 실행.

5. RDS로부터 데이터 전달하여 update.

### Input Data Preprocessing

- R_seq.shape = (n, 12, 1) : 12는 t-12 ~ t-1 <br>
R_seq.values.unique() = [0, 1, 2]

- H_seq.shape = (n, 4, 1) : 4는 w-4 ~ w-1 <br>
H_seq.values.unique() = ?, 일단 [0, 1, 2]로 가정.

- T.shape = (n, 3) : 3은 time_index, week_index, day_of_week

- S.shape = (n, 16) : sid(int) 포함 충전소 변수들

#### 고려할 점
- 우선, DB로부터 데이터를 가져와서 처리하는 것을 가정. <br>
이 때, sequence는 미리 잘라서 가져온다.

- 지속적 학습 및 배포를 수행한다고 하면, Data를 insert하는 것이 반드시 필요할텐데, <br>
raw Data를 한 번에 처리하여 DB에 전달하는 function을 만들어야 한다.

- 이 과정 수행 후, Baseline Model에 맞춰 lambda handler 수정

In [165]:
from private import db_info
import pandas as pd
import datetime
import pymysql
import torch

# get current datetime
def get_dt():
    now = datetime.datetime.now()
    dt = datetime.datetime(now.year, now.month, now.day, now.hour, now.minute//20*20, 00)
    str_dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt, str_dt

In [186]:
from private import db_info
import pandas as pd
import datetime
import pymysql
import torch


def db_connect():
    host, user, password, db = db_info()

    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        charset='utf8',
        port = 3306
    )

    return connection

# get time index
import numpy as np
def dt2T():
    connection = db_connect()
    select_query = "SELECT * FROM sequence LIMIT 1"
    with connection.cursor() as cursor:
        cursor.execute(select_query)
        result = cursor.fetchall()
        connection.close()

    dt, _ = get_dt()
    time_idx = dt.hour * 3 + dt.minute // 20
    weekday = dt.weekday()
    dow = weekday//5
    T = np.array([time_idx, weekday, dow]).reshape(1, -1)
    T = np.repeat(T, repeats = len(cursor.description)-1, axis = 0)
    return torch.tensor(T)
    

In [187]:
dt2T()

tensor([[59,  2,  0],
        [59,  2,  0],
        [59,  2,  0],
        ...,
        [59,  2,  0],
        [59,  2,  0],
        [59,  2,  0]], dtype=torch.int32)